In [6]:
import os
from PIL import Image
import sys 

import numpy as np
import matplotlib.pyplot as plt
import cv2

sys.path.append('..')
from utils.db_helper import get_files_paths_recursive

In [8]:
SOURCE_DIR = "C:/database/StyleGanv1"


In [80]:
def get_data(src_path: str)-> np.ndarray:
    images_paths = get_files_paths_recursive(src_path)
    images = []
    
    for image_path in images_paths:
        tmp_image = Image.open(image_path)
        np_image = np.array(tmp_image).astype(np.float32) / 255
        images.append(np_image)
        tmp_image.close()
    return np.array(images)


In [98]:
def dft_from_single_channel(np_channel: np.ndarray)-> np.ndarray:
        np_fft = np.fft.fft2(np_channel)
        np_fft = np.fft.fftshift(np_fft)
        np_fft = np.log(np.abs(np_fft)+1e-3)
        #normalize
        fft_min = np.percentile(np_fft,5)
        fft_max = np.percentile(np_fft,95)
        np_fft = (np_fft - fft_min)/(fft_max - fft_min)
        np_fft = (np_fft-0.5)*2
        np_fft[np_fft<-1] = -1
        np_fft[np_fft>1] = 1
        return np_fft

In [100]:
def dft_from_image(np_img: np.ndarray)-> np.ndarray:
    np_fft = np.empty(np_img.shape)
    if len(np_img.shape) == 3:
        for i in range(np_img.shape[-1]):
            np_fft[:,:,i] = dft_from_single_channel(np_img[:,:,i])
    elif len(np_img.shape) == 2:
        np_fft = dft_from_single_channel(np_img)
    else:
        raise ValueError('Bad shape of the image')
    return np_fft


In [101]:
def dft_from_dataset(src_path: str)-> np.ndarray:
    print("Loading dataset...")
    np_images = get_data(src_path)
    fft = []
    print("Processing database...")
    for x in np_images:
        fft.append(dft_from_image(x))
    np_fft = np.array(fft)